# Find feature = furnished

In [1]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 50) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?features=furnished&sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

In [2]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('../data/raw/furnished.json', 'w') as f:
    dump(property_metadata, f)

# find feature = unfurnished

In [3]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    if bs_object is not None:
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])



In [4]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/

with open('../data/raw/unfurnished.json', 'w') as f:
    dump(property_metadata, f)

In [19]:
import pandas as pd
df_furnished = pd.read_json('../data/raw/furnished.json')
df_raw = pd.read_json('../data/raw/unfurnished.json')

In [21]:
df_furnished = df_furnished.T
df_raw = df_raw.T

In [11]:
df_furnished = df_furnished.reset_index(drop=True)
df_raw = df_raw.reset_index(drop=True)

In [12]:
df_raw['Furnished'] = ' '

In [13]:
for i in range(0,len(df_raw)):
    for j in range(0,len(df_furnished)):
        if df_furnished['name'][j] == df_raw['name'][i]:
            df_raw['Furnished'][i] = 'True'
        else:
            df_raw['Furnished'][i] = 'False'

In [18]:
df_raw.tail()

,name,cost_text,coordinates,rooms,desc,Furnished
14,27 Moffat Street Brighton VIC 3186,"$3,200.00","[-37.9229499, 144.9973078]","[[4 Beds], [3 Baths], [2 Parking]]",Melissa Gilder,False
15,Brighton VIC 3186,Fine Furnished Residences at Westprecinct,"[-37.9043383, 144.9931825]","[[6 Beds], [4 Baths], [4 Parking]]",*Private inspections only at suggested times o...,False
16,7302/462 Elizabeth Street Melbourne VIC 3000,"$3,100 per week","[-37.807859, 144.960611]","[[3 Beds], [2 Baths], [1 Parking]]","class=""css-dxogle"">* Unverified feature<svg a...",False
17,9/9-11 Adamson Street Brighton VIC 3186,"$3,000 per week","[-37.9152565, 144.9991801]","[[3 Beds], [3 Baths], [2 Parking]]",Heidi Rodgers,False
18,2B Mulgoa Street Brighton VIC 3186,$3000 Per Week,"[-37.9025755, 144.988376]","[[4 Beds], [3 Baths], [2 Parking], []]",Samantha Middleton,False
